<a href="https://colab.research.google.com/github/yueannewang12/Spanner-rag/blob/main/graph_rag_agent_engine_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph RAG + Vertex AI Agent Engine Deployment Notebook

In [ ]:
# Install dependencies
%pip install --quiet google-cloud-aiplatform[adk,agent_engines]==1.121.0
%pip install --quiet google-cloud-spanner==3.56.0
%pip install --quiet langchain-google-spanner==0.8.2
%pip install --quiet langchain-google-vertexai==2.0.22


In [ ]:
from google.adk import Agent
from google.adk.tools import agent_tool, google_search
from google.adk.tools.function_tool import FunctionTool

from vertexai import agent_engines
import vertexai


In [ ]:
from langchain_google_spanner import SpannerGraphVectorContextRetriever

def use_node_vector_retriever2(
    question: str,
    graph_store,
    embedding_service,
    label_expr="Product",
    expand_by_hops=1,
):
    retriever = SpannerGraphVectorContextRetriever.from_params(
        graph_store=graph_store,
        embedding_service=embedding_service,
        label_expr=label_expr,
        expand_by_hops=expand_by_hops,
        top_k=1,
        k=10,
    )
    return retriever.invoke(question)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from langchain.schema.output_parser import StrOutputParser

SPANNERGRAPH_QA_TEMPLATE = """
You are a helpful retail assistant.
ONLY use the context; do not add knowledge.

Information:
Question: {question}
Graph Schema: {graph_schema}
Context: {context}

Helpful Answer:
"""

prompt = PromptTemplate(
    template=SPANNERGRAPH_QA_TEMPLATE,
    input_variables=["question", "graph_schema", "context"],
)

llm = ChatVertexAI(model="gemini-2.0-flash", temperature=0)
qa_chain = prompt | llm | StrOutputParser()

def ask_graph(query: str):
    from langchain_google_spanner import SpannerGraphStore

    INSTANCE = "properties"
    DATABASE = "graph-rag-colab"
    GRAPH = "my_graph"

    graph_store = SpannerGraphStore(
        instance_id=INSTANCE,
        database_id=DATABASE,
        graph_name=GRAPH,
    )

    embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

    context = use_node_vector_retriever2(
        question=query,
        graph_store=graph_store,
        embedding_service=embeddings,
        label_expr="Product",
        expand_by_hops=1,
    )

    answer = qa_chain.invoke(
        {
            "question": query,
            "graph_schema": graph_store.get_schema,
            "context": context,
        }
    )

    return {"result": answer}


In [ ]:
def ask_graph_tool_function(query: str):
    return ask_graph(query)

ask_graph_tool = FunctionTool(ask_graph_tool_function)
≈z

In [ ]:

search_agent = Agent(
    model="gemini-2.5-flash",
    name="search_agent",
    instruction="External lookup expert.",
    tools=[google_search],
)

root_agent = Agent(
    name="graph_rag_agent",
    model="gemini-2.5-flash",
    instruction="""
You are a retail assistant using a Google Spanner product graph.
Always use ask_graph first.
""",
    tools=[
        agent_tool.AgentTool(agent=search_agent),
        ask_graph_tool,
    ],
)


In [ ]:
app = agent_engines.AdkApp(
    app_name="graph_rag_agent_app",
    agent=root_agent,
    enable_tracing=True,
)

In [ ]:


GCP_PROJECT_ID = "anne-test-project1"   # <-- your actual project ID
REGION = "us-central1"                  # <-- your region

GCP_PROJECT_NUMBER = "198925083406"

STAGING_BUCKET = f"gs://{GCP_PROJECT_ID}-vertexai-staging"

vertexai.init(
    project=GCP_PROJECT_ID,
    location=REGION,
    staging_bucket=STAGING_BUCKET,
)

remote_app = agent_engines.create(
    display_name="graph_rag_agent_app",
    agent_engine=app,
    requirements=[
        "google-cloud-aiplatform[adk]>=1.126.1",
        "google-cloud-spanner==3.56.0",
        "langchain-google-spanner==0.8.2",
        "langchain-google-vertexai==2.0.22",
        "langchain-experimental==0.3.4",
        "pydantic==2.12.5",
        "cloudpickle==3.1.1",
    ]
)


print("Resource Name:", remote_app.resource_name)


Push the code to Colab